# Sentiment Analysis

This is my first project regarding NLP. The goal with this project is to learn how to perform Sentiment Analysis from scratch, and for that reason I will try to use as fewer libraries as possible.

The dataset that I will be using is the following from Kaggle: https://www.kaggle.com/datasets/alexandrakim2201/spotify-dataset/data





In [6]:
import numpy as np
import pandas as p

import os

# load the dataset
filename = "/kaggle/input/spotify-dataset/DATASET.csv"
df = pd.read_csv(filename)
print(df.shape)
df.head(10)

(52702, 2)


,Review,label
0,"Great music service, the audio is high quality...",POSITIVE
1,Please ignore previous negative rating. This a...,POSITIVE
2,"This pop-up ""Get the best Spotify experience o...",NEGATIVE
3,Really buggy and terrible to use as of recently,NEGATIVE
4,Dear Spotify why do I get songs that I didn't ...,NEGATIVE
5,The player controls sometimes disappear for no...,NEGATIVE
6,I love the selection and the lyrics are provid...,POSITIVE
7,Still extremely slow when changing storage to ...,NEGATIVE
8,It's a great app and the best mp3 music app I ...,POSITIVE
9,"I'm deleting this app, for the following reaso...",NEGATIVE


## Preprocessing

After loading the dataset, we should do some preprocessing. The first thing that we are going to do is some general EDA and then proceed to process the text.

### EDA

Lets check for missing and duplicated values


In [11]:
print("Null:")
df.isnull().sum()


Null:


Review    16
label      0
dtype: int64

In [20]:
print("Duplicate:")
df.duplicated().sum()

Duplicate:


1229

We can see that there are 16 null entries, but also 1229 duplicated ones.